In [1]:
import pandas as pd
import numpy as np

In [2]:
bottle_csv_path = "/iliad/u/jmeribe/projects/vlm-scaffolding/tracking_results_bottle_with_my_policies.csv"
bottle_csv_with_base = "/iliad/u/jmeribe/projects/vlm-scaffolding/tracking_results_bottle_with_vincent_policies.csv"

bottle_df = pd.read_csv(bottle_csv_path)

bottle_df

,Unnamed: 0,method,environment,success,num_envs,run_name,model_name,split,distribution
0,0,scripted,EnvBottle-v0-phys-rand-with-cond,0.083008,1024,irrelevant,model_1999.pt,test,"beta(a=6.1, b=50.0)"
1,1,scripted,EnvBottle-v0-phys-rand,0.151367,1024,irrelevant,model_1999.pt,test,"beta(a=6.1, b=50.0)"
2,2,scripted,EnvBottle-v0-phys-rand-with-cond,0.083984,1024,irrelevant,model_1999.pt,test,"beta(a=6.1, b=50.0)"
3,3,scripted,EnvBottle-v0-phys-rand,0.132812,1024,irrelevant,model_1999.pt,test,"beta(a=6.1, b=50.0)"
4,4,scripted,EnvBottle-v0-phys-rand-with-cond,0.000000,1024,irrelevant,model_1999.pt,test,"beta(a=50, b=6.1)"
...,...,...,...,...,...,...,...,...,...
95,95,scripted,EnvBottle-v0-phys-rand,0.215820,1024,irrelevant,model_250.pt,test,"beta(a=2.5, b=6.8)"
96,96,scripted,EnvBottle-v0-phys-rand-with-cond,0.070312,1024,irrelevant,model_250.pt,train1,"beta(a=2.5, b=1.9)"
97,97,scripted,EnvBottle-v0-phys-rand,0.090820,1024,irrelevant,model_250.pt,train1,"beta(a=2.5, b=1.9)"
98,98,scripted,EnvBottle-v0-phys-rand-with-cond,0.083984,1024,irrelevant,model_250.pt,train1,"beta(a=2.5, b=6.8)"


In [3]:
# bottle_df_with_base = pd.read_csv(bottle_csv_with_base)

# General Performance: 
### conditioned policy narrowly wins
### original policy loses by 100%

In [4]:
valid_bottle_no_cond_df = bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvBottle-v0-phys-rand")]["success"]

general_no_cond = valid_bottle_no_cond_df.to_numpy().mean()
general_no_cond

0.1254296875

In [5]:
valid_bottle_cond_df = bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvBottle-v0-phys-rand-with-cond")]["success"]

general_cond = valid_bottle_cond_df.to_numpy().mean()
general_cond

0.106015625

In [6]:
# bottle_df_vincent = bottle_df_with_base[(bottle_df_with_base['num_envs'] > 1000)]["success"]

# general_no_cond_vincent = bottle_df_vincent.to_numpy().mean()
# general_no_cond_vincent

# Performance Across Different Epochs
### conditioned policy loses 2/5

In [7]:
epoch_names = bottle_df["model_name"].unique()
epoch_names

array(['model_1999.pt', 'model_1750.pt', 'model_1250.pt', 'model_750.pt',
       'model_250.pt'], dtype=object)

In [8]:
epoch_perf_cond = []
epoch_perf_no_cond = []

for epoch in epoch_names:
    epoch_perf_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvBottle-v0-phys-rand-with-cond") & (bottle_df["model_name"] == epoch)]["success"].to_numpy().mean()
    )
    epoch_perf_no_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvBottle-v0-phys-rand") & (bottle_df["model_name"] == epoch)]["success"].to_numpy().mean()
    )

epoch_dict_cond = dict(zip(epoch_names, epoch_perf_cond))

epoch_dict_cond

{'model_1999.pt': 0.12451171875,
 'model_1750.pt': 0.176171875,
 'model_1250.pt': 0.0904296875,
 'model_750.pt': 0.08876953125,
 'model_250.pt': 0.0501953125}

In [9]:
epoch_dict_un_cond = dict(zip(epoch_names, epoch_perf_no_cond))

epoch_dict_un_cond

{'model_1999.pt': 0.186328125,
 'model_1750.pt': 0.14443359375,
 'model_1250.pt': 0.12353515625,
 'model_750.pt': 0.09521484375,
 'model_250.pt': 0.07763671875}

In [10]:
wins = 0
games = 0

for key, value in epoch_dict_cond.items():
    wins += 1 if value > epoch_dict_un_cond[key] else 0
    games += 1

print(f"{wins}/{games}")

1/5


# Train and Test
### Conditioned policy wins on train loses on test

In [11]:
dataset_perf_cond = []
dataset_perf_no_cond = []

datasets = ["train1", "test"]

for dataset in datasets:
    dataset_perf_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvBottle-v0-phys-rand-with-cond") & (bottle_df["split"] == dataset)]["success"].to_numpy().mean()
    )
    dataset_perf_no_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvBottle-v0-phys-rand") & (bottle_df["split"] == dataset)]["success"].to_numpy().mean()
    )

dict(zip(datasets, dataset_perf_cond))

{'train1': 0.11430220170454546, 'test': 0.09950474330357142}

In [12]:
dict(zip(datasets, dataset_perf_no_cond))

{'train1': 0.13303444602272727, 'test': 0.11945452008928571}

# Distribution
### Conditioned policy loses 1/5

In [13]:
distribution_names = bottle_df["distribution"].unique()
distribution_names

array(['beta(a=6.1, b=50.0)', 'beta(a=50, b=6.1)', 'beta(a=50, b=50)',
       'beta(a=2.5, b=1.9)', 'beta(a=2.5, b=6.8)'], dtype=object)

In [14]:
distribution_perf_cond = []
distribution_perf_no_cond = []

for distribution in distribution_names:
    distribution_perf_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvBottle-v0-phys-rand-with-cond") & (bottle_df["distribution"] == distribution)]["success"].to_numpy().mean()
    )
    distribution_perf_no_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvBottle-v0-phys-rand") & (bottle_df["distribution"] == distribution)]["success"].to_numpy().mean()
    )

distribution_dict_cond =  dict(zip(distribution_names, distribution_perf_cond))

In [15]:
distribution_dict_un_cond = dict(zip(distribution_names, distribution_perf_no_cond))

In [16]:
wins = 0
games = 0

for key, value in distribution_dict_cond.items():
    wins += 1 if value > distribution_dict_un_cond[key] else 0
    games += 1

print(f"{wins}/{games}")

1/5


# Conclusions
1. On basically every hyperparameter set when we don't condition the policy on the physical parameters of object being manipulated we perform better.

2. This doesn't make sense as we would expect the unconditioned policy to act as a lower bound for the performance of the conditioned policy.

3. However, when we compare average train and test performance we see that the conditioned policy performs better on train set and worse on the test set.

4. Indicating, that the conditioned policy is probably overfitting and may just be harder to train, given the setup.

5. Addtionally, the slim margins and generally low performance that we observe indicate that the scaffolding framework is not effective for tasks with high entropy physical property distributions or tasks where physicaly properties are important for performance, because it does not seem like it is effectively using them in this case.